In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
from torchvision import transforms
from torch.utils.data import Dataset 
import numpy as np
from PIL import Image
import os
import nibabel as nib
import re
import pandas as pd

In [3]:
np.random.seed(12345)

## Cuda Check

In [4]:
print('avaliable:\t', torch.cuda.is_available())
print('current:\t', torch.cuda.current_device())
print('device: \t', torch.cuda.device(0))
print('count:\t\t', torch.cuda.device_count())
print('name:\t\t', torch.cuda.get_device_name(0))

avaliable:	 True
current:	 0
device: 	 <torch.cuda.device object at 0x0000023D00276708>
count:		 1
name:		 NVIDIA GeForce GTX 1060 6GB


## Input Data

###### Jimi's Path

In [5]:
path = 'D:/BT_Classification/backup/train/'

## Data Augmenting

In [6]:
def load_image(path):
    """
    Parameters:
        path: path to file
    Return:
        img: image tensor
    """
    img_nii = nib.load(path)
    
    img_np = np.squeeze(img_nii.get_fdata(dtype=np.float32))
    
    return torch.from_numpy(img_np)

In [7]:
def check_crop_dim(full_size, crop_size):

    if full_size[0] < crop_size[0] or full_size[1] < crop_size[1] or full_size[2] < crop_size[2]:
        return True

    return False

In [8]:
def get_valid_crop(image_tensor, crop_size):
#     print(image_tensor.shape)
    mask = np.where(image_tensor < torch.mean(image_tensor), 0, image_tensor)
    mask[mask > 0] = 1
#     print(mask.shape)
    c_s, c_w, c_h = crop_size
    f_s, f_w, f_h = image_tensor.shape
    
    total = np.sum(mask)
    
    search = True
    while search:
        if f_s == c_s:
            x = 0
        else:
            x = np.random.randint(f_s - c_s)

        if f_w == c_w:
            y = 0
        else:
            y = np.random.randint(f_w - c_w)

        if f_h == c_h:
            z = 0
        else:
            z = np.random.randint(f_h - c_h)

        cropped = mask[x:x + c_s, y:y + c_w, z:z + c_h]
        
        c_total = np.sum(cropped)
        
        if (c_total / total) > 0.1:
            search = False
    
    return image_tensor[x:x + c_s, y:y + c_w, z:z + c_h]
        
            
                

In [9]:
def get_random_crops(path, crop_size, count, label_path):
    if type(path) == str:
        path = [path]
    
    labels = pd.read_csv(label_path, dtype={'BraTS21ID':str, 'MGMT_value':np.int64})
#     print(labels.head())
    cropped = []
    
    for i, p in enumerate(path):
        print(i, p)
        # SKIP BECAUSE SOMETHING WRONG
        if '00053' in p:
            print('\tSkipped')
            continue
        if '00186' in p:
            print('\tSkipped')
            continue
            
        # SKIP BECAUSE HOST SAID TO
        if '00109' in p:
            print('\tSkipped')
            continue
        if '00123' in p:
            print('\tSkipped')
            continue
        if '00709' in p:
            print('\tSkipped')
            continue
        
        
        image_tensor = load_image(p)
        p_id = re.split('\\\\|/', p)[-2]
#         cropped.append([image_tensor, labels.loc[labels['BraTS21ID'] == p_id, 'MGMT_value'].iloc[0]])
#         print(p_id)

        if check_crop_dim(image_tensor.shape, crop_size):
            print('\tCrop size too large. Image:{} \t crop:{}'.format(image_tensor.shape, crop_size))
            continue

        for i in range(count):
            cropped.append([get_valid_crop(image_tensor, crop_size), labels.loc[labels['BraTS21ID'] == p_id, 'MGMT_value'].iloc[0]])
    
    data = torch.utils.data.DataLoader(cropped)
    return cropped, data
    
    

In [ ]:
crops, t_data = get_random_crops(path=path+'00000\FLAIR.nii', crop_size=(70, 70, 70), count=1, label_path=path+'/../../train_labels.csv')
print(crops)
print(t_data)

for i, data in enumerate(t_data, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    
    print(inputs.shape)
    print(inputs.unsqueeze(-1).shape)
    print(inputs.unsqueeze(-1))

### View Croppings

In [10]:
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt

%matplotlib ipympl

In [11]:
def view(image):
    def axial(ax_slice):
        return image[ax_slice,:,:]

    def cor(cor_slice):
        return image[:,cor_slice,:]

    def sag(sag_slice):
        return image[:,:,sag_slice]

    # define layout
    fig, ax = plt.subplots(3,1)

    for a in ax:
        a.xaxis.set_visible(False)
        a.yaxis.set_visible(False)

    ctrl1 = iplt.imshow(axial, ax_slice=np.arange(image.shape[2]), aspect='auto', ax=ax[0], vmin=0, vmax=1, cmap ='gray')
    ctrl2 = iplt.imshow(sag, sag_slice=np.arange(image.shape[0]), aspect='auto', ax=ax[1], vmin=0, vmax=1, cmap ='gray')
    crtl3 = iplt.imshow(cor, cor_slice=np.arange(image.shape[1]), aspect='auto', ax=ax[2], vmin=0, vmax=1, cmap ='gray' )
    fig.tight_layout()


In [12]:
view(load_image(path+'00053/FLAIR.nii'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
view(load_image(path+'00186/FLAIR.nii'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
view(crops[0])

In [ ]:
view(crops[1])

## Building the 3D CNN and MLP

In [14]:
import torch.nn as nn

In [ ]:
image_tensor = load_image(path+'00000/FLAIR.nii')

In [ ]:
image_tensor.shape

In [ ]:
print(image_tensor[:2])

In [ ]:
tensor = torch.unsqueeze(image_tensor, -1)
print(tensor.shape)
print(tensor[:2])

In [ ]:
test = nn.Conv3d(1, 5, 5, bias=False)

In [ ]:
out = test(image_tensor.unsqueeze(0).unsqueeze(0))
print(out.shape)

In [ ]:
def make_nn_model():
    in_channel = 1
    base_out = 4
    return nn.Sequential(nn.Conv3d(in_channels=in_channel, out_channels=base_out, kernel_size=3, bias=False),
                         nn.LeakyReLU(),
                         nn.Dropout3d(p=0.3),
                         nn.Conv3d(base_out, base_out*2, 3, bias=False),
                         nn.InstanceNorm3d(base_out*2),
                         nn.LeakyReLU(),
                         nn.Dropout3d(p=0.3),
                         nn.Conv3d(base_out*2, base_out*4, 3, bias=False),
                         nn.InstanceNorm3d(base_out*4),
                         nn.LeakyReLU(),
                         nn.Conv3d(base_out*4, base_out*8, 3, bias=False),
                         nn.MaxPool3d(2),
                         nn.Dropout3d(p=0.3),
                         
                         
                         nn.Conv3d(base_out*8, base_out*8, 3, bias=False),
                         nn.InstanceNorm3d(base_out*8),
                         nn.LeakyReLU(),
                         nn.Dropout3d(p=0.3),
                         nn.Conv3d(base_out*8, base_out*16, 3, bias=False),
                         nn.LeakyReLU(),
                         nn.MaxPool3d(2),
                         nn.Dropout3d(p=0.3),
                         
                         
                         nn.Conv3d(base_out*16, base_out*16, 3, bias=False),
                         nn.InstanceNorm3d(base_out*16),
                         nn.LeakyReLU(),
                         nn.Dropout3d(p=0.3),
                         nn.Conv3d(base_out*16, base_out*32, 3, bias=False),
                         nn.InstanceNorm3d(256),
                         nn.LeakyReLU(),
                         nn.Dropout3d(p=0.3),
                        )

In [ ]:
def make_lin_model():
    return nn.Sequential(nn.Linear(10616832, 64),
#                          nn.Dropout2d(p=0.2),
#                          nn.Linear(2048, 64),
                         nn.Dropout2d(p=0.2),
                         nn.Linear(64, 8),
                         nn.Dropout2d(p=0.2),
                         nn.Linear(8, 2)
                        )

In [ ]:
nn_model = make_nn_model()

In [ ]:
out = nn_model(image_tensor.unsqueeze(0).unsqueeze(0))

In [ ]:
print(out.shape)

In [ ]:
out = torch.flatten(out,1)

In [ ]:
print(out.shape)

In [ ]:
lin_model = make_lin_model()

In [ ]:
out = lin_model(out)

In [ ]:
print(out.shape)

In [ ]:
print(out)

## Create Loss Fxn and Optimizer

## Testing on First 100 data

In [15]:
import glob

In [16]:
files = glob.glob(path+'000**/FLAIR.nii')
files.extend(glob.glob(path+'001**/FLAIR.nii'))

In [17]:
print(files)

['D:/BT_Classification/backup/train\\00000\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00002\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00003\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00005\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00006\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00008\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00009\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00011\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00012\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00014\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00017\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00018\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00019\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00020\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00021\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00022\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00024\\FLAIR.nii', 'D:/BT_Classification/backup/train\\00025\\FLAIR.nii', 'D:/BT_Cl

In [18]:
crops, train_data = get_random_crops(files, crop_size=(50, 50, 50), count=1, label_path=path+'/../../train_labels.csv')

0 D:/BT_Classification/backup/train\00000\FLAIR.nii
1 D:/BT_Classification/backup/train\00002\FLAIR.nii
2 D:/BT_Classification/backup/train\00003\FLAIR.nii
3 D:/BT_Classification/backup/train\00005\FLAIR.nii
4 D:/BT_Classification/backup/train\00006\FLAIR.nii
5 D:/BT_Classification/backup/train\00008\FLAIR.nii
6 D:/BT_Classification/backup/train\00009\FLAIR.nii
7 D:/BT_Classification/backup/train\00011\FLAIR.nii
8 D:/BT_Classification/backup/train\00012\FLAIR.nii
9 D:/BT_Classification/backup/train\00014\FLAIR.nii
10 D:/BT_Classification/backup/train\00017\FLAIR.nii
11 D:/BT_Classification/backup/train\00018\FLAIR.nii
12 D:/BT_Classification/backup/train\00019\FLAIR.nii
13 D:/BT_Classification/backup/train\00020\FLAIR.nii
14 D:/BT_Classification/backup/train\00021\FLAIR.nii
15 D:/BT_Classification/backup/train\00022\FLAIR.nii
16 D:/BT_Classification/backup/train\00024\FLAIR.nii
17 D:/BT_Classification/backup/train\00025\FLAIR.nii
18 D:/BT_Classification/backup/train\00026\FLAIR.nii
19 

In [19]:
print(len(crops))
print(len(train_data))

129
129


In [25]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_channel = 1
        self.base_channel = 4
        
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels=self.in_channel, out_channels=self.base_channel, kernel_size=3, bias=False),
            nn.LeakyReLU(),
            nn.Dropout3d(p=0.3),
            nn.Conv3d(self.base_channel, self.base_channel*2, 3, bias=False),
            nn.InstanceNorm3d(self.base_channel*2),
            nn.LeakyReLU(),
            nn.Dropout3d(p=0.3),
            nn.Conv3d(self.base_channel*2, self.base_channel*4, 3, bias=False),
            nn.InstanceNorm3d(self.base_channel*4),
            nn.LeakyReLU(),
            nn.Dropout3d(p=0.3),
            nn.Conv3d(self.base_channel*4, self.base_channel*8, 3, bias=False),
            nn.MaxPool3d(2),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(self.base_channel*8, self.base_channel*8, 3, bias=False),
            nn.InstanceNorm3d(self.base_channel*8),
            nn.LeakyReLU(),
            nn.Dropout3d(p=0.3),
            nn.Conv3d(self.base_channel*8, self.base_channel*16, 3, bias=False),
            nn.LeakyReLU(),
            nn.MaxPool3d(2),
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv3d(self.base_channel*16, self.base_channel*16, 3, bias=False),
            nn.InstanceNorm3d(self.base_channel*16),
            nn.LeakyReLU(),
            nn.Dropout3d(p=0.3),
            nn.Conv3d(self.base_channel*16, self.base_channel*32, 3, bias=False),
            nn.InstanceNorm3d(self.base_channel*32),
            nn.LeakyReLU(),
        )
        
        self.lin = nn.Sequential(
            nn.Linear(8192, 64),
#             nn.Dropout2d(p=0.2),
#             nn.Linear(2048, 64),
            nn.Dropout2d(p=0.2),
            nn.Linear(64, 8),
            nn.Dropout2d(p=0.2),
            nn.Linear(8, 2)
        )
    
    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(out1)
        out3 = self.conv3(out2)
        
        out3 = torch.flatten(out3,1)
        
        out4 = self.lin(out3)
        
        return out4

In [26]:
net = Net()

In [27]:
import torch.optim as optim

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [35]:
if torch.cuda.is_available():
    dev = "cuda:0" 
else:
    dev = 'cpu'

net.to('cpu')

for epoch in range(2):  # loop over the dataset multiple times

    for i, data in enumerate(train_data, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.unsqueeze(1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' %
          (epoch + 1, loss.item()))
        
    torch.save({
        'epoch': epoch,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, path+'/../../checkpoints/cp_{}.cp'.format(epoch))

print('Finished Training')
torch.save(net.state_dict(), path+'/../model')

NotImplementedError: Could not run 'aten::slow_conv3d_forward' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::slow_conv3d_forward' is only available for these backends: [CPU, BackendSelect, Named, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, UNKNOWN_TENSOR_TYPE_ID, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, Autocast, Batched, VmapMode].

CPU: registered at aten\src\ATen\RegisterCPU.cpp:16286 [kernel]
BackendSelect: fallthrough registered at ..\aten\src\ATen\core\BackendSelectFallbackKernel.cpp:3 [backend fallback]
Named: registered at ..\aten\src\ATen\core\NamedRegistrations.cpp:7 [backend fallback]
ADInplaceOrView: fa